**Project 2.5 Group 1**   
Big Collection of procedures code and notes

**HEADING (Put this at the beginning of every procedure)**

In [ ]:
-- =============================================
-- Author:        FIRSTNAME LASTNAME
-- Create date:   2025-05-03
-- Description:   [WHAT DOES THIS DO]
-- Object type:   [PICK 1 (what is this?): Stored Procedure | Function | View | Table | Trigger]
-- Usage:         [How or where to call this]
-- Parameters:    [@UserAuthorizationKey INT + others]
-- Change History:
--   YYYY-MM-DD   FIRSTNAME LASTNAME   Initial creation
--   YYYY-MM-DD   [Your Name]   [Description of change]
-- =============================================

In [1]:
-- =============================================
-- Author:        Jaskaran Bains
-- Create date:   2025-05-03
-- Description:   [Creates schema to place the procedures of workflow]
-- Object type:   [Single Line Query]
-- Usage:         [Call before running the backing up stored procedure]
-- Parameters:    [N/A]
-- Change History:
--   YYYY-MM-DD   Jaskaran Bains   Initial creation
-- =============================================
CREATE SCHEMA Group1;

: Msg 2714, Level 16, State 6, Line 11
There is already an object named 'Group1' in the database.

: Msg 2759, Level 16, State 0, Line 11
CREATE SCHEMA failed due to previous errors.

Total execution time: 00:00:00.020

In [2]:
-- =============================================
-- Author:        Jaskaran Bains
-- Create date:   2025-05-03
-- Description:   [Creates UserAuthorizationTable]
-- Object type:   [Stored Procedure]
-- Usage:         [Call before running the backing up stored procedure + run as exec]
-- Parameters:    [N/A]
-- Change History:
-- 2025-05-04   Jaskaran Bains   Initial creation
-- =============================================
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE PROCEDURE [Group1].[CreateUserAuthorizationTable]
AS
BEGIN
    SET NOCOUNT ON;

    IF EXISTS (SELECT 1 FROM INFORMATION_SCHEMA.TABLES 
               WHERE TABLE_SCHEMA = 'Group1' AND TABLE_NAME = 'UserAuthorizationTable')
    BEGIN
        DROP TABLE [Group1].[UserAuthorizationTable];
    END;

    
    CREATE TABLE [Group1].[UserAuthorizationTable] (
        UserAuthorizationKey INT NOT NULL PRIMARY KEY,
        ClassTime NCHAR(5) NULL CHECK (ClassTime IN ('10:45')),
        IndividualProject NVARCHAR(60) NULL DEFAULT 'PROJECT 2.5 Prestige Cars',
        GroupMemberLastName NVARCHAR(35) NOT NULL,
        GroupMemberFirstName NVARCHAR(25) NOT NULL,
        GroupName NVARCHAR(20) NOT NULL,
        DateAdded DATETIME2 NULL DEFAULT SYSDATETIME(),
        DateOfLastUpdate DATETIME2 NULL DEFAULT SYSDATETIME()
    );

    INSERT INTO [Group1].[UserAuthorizationTable] (
        UserAuthorizationKey,
        ClassTime,
        IndividualProject,
        GroupMemberLastName,
        GroupMemberFirstName,
        GroupName,
        DateAdded,
        DateOfLastUpdate
    )
    VALUES
        (1, '10:45', 'Normalize, Backup, TrackWorkFLow, Clean, Truncate', 'Bains', 'Jaskaran', 'Group#1', DEFAULT, DEFAULT),
        (2, '10:45', 'Country, Customer, UDT', 'Ahamed', 'Sabir', 'Group#1', DEFAULT, DEFAULT),
        (3, '10:45', 'Make, Sale, UDT', 'Cristobal', 'David', 'Group#1', DEFAULT, DEFAULT),
        (4, '10:45', 'Mode, UDT', 'Maysoon', 'Zhian', 'Group#1', DEFAULT, DEFAULT),
        (5, '10:45', 'Stock, UDT', 'Weng Jiang', 'Bryan', 'Group#1', DEFAULT, DEFAULT),
        (6, '10:45', 'Detail, UDT', 'Zheng', 'Zheng', 'Group#1', DEFAULT, DEFAULT);

END;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 2714, Level 16, State 3, Procedure CreateUserAuthorizationTable, Line 1
There is already an object named 'CreateUserAuthorizationTable' in the database.

Total execution time: 00:00:00.019

In [4]:
-- =============================================
-- Author:        Jaskaran Bains
-- Create date:   2025-05-03
-- Description:   Creates table to store workflow steps
-- Object type:   Stored Procedure
-- Usage:         Call before running the backup stored procedure + run as exec
-- Parameters:    N/A
-- Change History:
-- 2025-05-04   Jaskaran Bains   Initial creation
-- 2025-05-18   [Your Name]      Added IDENTITY to WorkFlowStepKey
-- =============================================
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
create PROCEDURE [Group1].[CreateWorkFlowTable]
AS
BEGIN
    SET NOCOUNT ON;

    CREATE TABLE [Group1].[WorkflowSteps] (
        WorkFlowStepKey INT IDENTITY(1,1) NOT NULL PRIMARY KEY,
        WorkFlowStepDescription NVARCHAR(100) NOT NULL,
        WorkFlowStepTableRowCount INT NULL DEFAULT 0,
        StartingDateTime DATETIME2 NULL DEFAULT SYSDATETIME(),
        EndingDateTime DATETIME2 NULL DEFAULT SYSDATETIME(),
        ClassTime VARCHAR(5) NULL DEFAULT '10:45',
        UserAuthorizationKey INT NOT NULL,
        CONSTRAINT FK_WorkflowSteps_UserAuthorization FOREIGN KEY (UserAuthorizationKey) 
            REFERENCES Group1.UserAuthorizationTable (UserAuthorizationKey)
    );

END;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.057

In [2]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author:        Jaskaran Bains
-- Create date:   2025-05-03
-- Description:   [Creates the function that tracks work flows and inserts them into the tabls]
-- Object type:   [Stored Procedure]
-- Usage:         [Call before running the backing up stored procedure + run as exec]
-- Parameters:    [N/A]
-- Change History:
-- 2025-05-04   Jaskaran Bains   Initial creation
-- =============================================
CREATE PROCEDURE [Group1].[TrackWorkFlow]
    @StartTime DATETIME2,
    @WorkFlowDescription NVARCHAR(100),
    @WorkFlowStepTableRowCount INT,
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    -- Insert workflow step details
    INSERT INTO [Group1].[WorkflowSteps] (
        WorkFlowStepDescription,
        WorkFlowStepTableRowCount,
        StartingDateTime,
        EndingDateTime,
        ClassTime,
        UserAuthorizationKey
    )
    VALUES (
        @WorkFlowDescription,
        @WorkFlowStepTableRowCount,
        @StartTime,
        SYSDATETIME(),
        (SELECT ClassTime FROM [Group1].[UserAuthorizationTable] WHERE UserAuthorizationKey = @UserAuthorizationKey),
        @UserAuthorizationKey
    );
END;
GO


Commands completed successfully.

Total execution time: 00:00:00.020

In [18]:
-- =============================================
-- Author:        Jaskaran Bains
-- Create date:   2025-05-03
-- Description:   [Creates schema to place the backup of Data schema]
-- Object type:   [Single Line Query]
-- Usage:         [Call before running the backing up stored procedure]
-- Parameters:    [N/A]
-- Change History:
--   YYYY-MM-DD   Jaskaran Bains   Initial creation
-- =============================================
CREATE SCHEMA BackupData;

: Msg 2714, Level 16, State 6, Line 11
There is already an object named 'BackupData' in the database.

: Msg 2759, Level 16, State 0, Line 11
CREATE SCHEMA failed due to previous errors.

Total execution time: 00:00:00.029

In [8]:
-- Uncomment to test
drop table 
[BackupData].Country, 
[BackupData].Customer, 
[BackupData].Make, 
[BackupData].Model, 
[BackupData].Sale, 
[BackupData].SaleDetail, 
[BackupData].Stock;
-- for testing purposes

Commands completed successfully.

Total execution time: 00:00:00.036

In [7]:
-- =============================================
-- Author:        Jaskaran Bains
-- Create date:   2025-05-03
-- Description:   Creates a full backup of all tables in the [Data] schema into [BackupData] schema
-- Object type:   Stored Procedure
-- Usage:         EXEC [BackupData].[BackupDataSchema] @UserAuthorizationKey = 1;
-- Parameters:    @UserAuthorizationKey INT
-- Change History:
-- 2025-05-03   Your Full Name   Initial creation
-- =============================================

Create PROCEDURE [BackupData].[BackupDataSchema]
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'BackupData')
    EXEC('CREATE SCHEMA BackupData');

    -- Backup Country
    SELECT * INTO [BackupData].[Country]
    FROM [Data].[Country];

    EXEC sp_rename 'BackupData.Country.SalesRegion', 'SaleRegion', 'COLUMN';
    ALTER TABLE [BackupData].[Country]
    DROP COLUMN CountryFlag, FlagFileName, FlagFileType;

    -- Backup Customer
    SELECT * INTO [BackupData].[Customer]
    FROM [Data].[Customer];

    EXEC sp_rename 'BackupData.Customer.PostCode', 'PostalCode', 'COLUMN';

    -- Backup Make
    SELECT * INTO [BackupData].[Make]
    FROM [Data].[Make];

    -- Backup Model
    SELECT * INTO [BackupData].[Model]
    FROM [Data].[Model];

    -- Backup Sale
    SELECT * INTO [BackupData].[Sale]
    FROM [Data].[Sales];

    EXEC sp_rename 'BackupData.Sale.SalesID', 'SaleID', 'COLUMN';
    EXEC sp_rename 'BackupData.Sale.TotalSalePrice', 'SaleTotal', 'COLUMN';
    ALTER TABLE [BackupData].[Sale]
    DROP COLUMN ID, SaleDate;

    -- Backup SaleDetail
    SELECT * INTO [BackupData].[SaleDetail]
    FROM [Data].[SalesDetails];

    EXEC sp_rename 'BackupData.SaleDetail.SalesDetailsID', 'SaleDetailID', 'COLUMN';
    EXEC sp_rename 'BackupData.SaleDetail.SalesID', 'SaleID', 'COLUMN';

    ALTER TABLE [BackupData].[SaleDetail]
    ADD DateBought DATE, TimeBought TIME;

    UPDATE sd
    SET 
        sd.DateBought = s.DateBought,
        sd.TimeBought = s.TimeBought
    FROM [BackupData].[SaleDetail] sd
    JOIN [Data].[Stock] s ON sd.StockID = s.StockCode;

    -- Backup Stock
    SELECT * INTO [BackupData].[Stock]
    FROM [Data].[Stock];

    EXEC sp_rename 'BackupData.Stock.StockCode', 'StockID', 'COLUMN';
    EXEC sp_rename 'BackupData.Stock.RepairsCost', 'RepairCost', 'COLUMN';
    EXEC sp_rename 'BackupData.Stock.PartsCost', 'PartCost', 'COLUMN';
    EXEC sp_rename 'BackupData.Stock.TransportInCost', 'TransportCost', 'COLUMN';
    EXEC sp_rename 'BackupData.Stock.BuyerComments', 'BuyerComment', 'COLUMN';

    DECLARE @WorkFlowStepTableRowCount INT;
SET @WorkFlowStepTableRowCount = @@ROWCOUNT;

-- Log workflow step
EXEC [Group1].[TrackWorkFlow]
        @StartTime,
        'Backed up all of [Data] to [BackupData]',
        @WorkFlowStepTableRowCount,
        @UserAuthorizationKey;

END;

: Msg 2714, Level 16, State 3, Procedure BackupDataSchema, Line 12
There is already an object named 'BackupDataSchema' in the database.

Total execution time: 00:00:00.019

In [5]:
-- =============================================
-- Author:        Jaskaran Bains
-- Create date:   2025-05-03
-- Description:   [Drops FKs and Tables in Data Schema]
-- Object type:   [Script]
-- Usage:         [Exec to drop fk and truncate tables]
-- Parameters:    [NA]
-- Change History:
-- YYYY-MM-DD   Jaskaran Bains   Initial creation
-- =============================================


ALTER TABLE [Data].[Model] DROP CONSTRAINT IF EXISTS [FK_Model_Make];
ALTER TABLE [Data].[Make] DROP CONSTRAINT IF EXISTS [FK_Make_Country];
ALTER TABLE [Data].[Country] DROP CONSTRAINT IF EXISTS [FK_Country_SalesRegion];
ALTER TABLE [Data].[Customer] DROP CONSTRAINT IF EXISTS [FK_Customer_Country];
ALTER TABLE [Data].[SalesDetails] DROP CONSTRAINT IF EXISTS [FK_SalesDetails_Sales];
ALTER TABLE [Data].[SalesDetails] DROP CONSTRAINT IF EXISTS [FK_SalesDetails_Stock];
ALTER TABLE [Data].[Sales] DROP CONSTRAINT IF EXISTS [FK_Sales_Customer];
ALTER TABLE [Data].[Stock] DROP CONSTRAINT IF EXISTS [FK_Stock_Model];
GO

-- Truncate tables in the Data schema
TRUNCATE TABLE [Data].[SalesRegion];
TRUNCATE TABLE [Data].[Country];
TRUNCATE TABLE [Data].[Customer];
TRUNCATE TABLE [Data].[Make];
TRUNCATE TABLE [Data].[Model];
TRUNCATE TABLE [Data].[SalesDetails];
TRUNCATE TABLE [Data].[Stock];
TRUNCATE TABLE [Data].[Sales];
GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.127

In [16]:

-- NOMRMALIZAITON OF DATA BASE VIA COMPILED CODE FROM GROUP
-- ALTER TABLES BY DROPPING THE COLUMNS NOT USED AND OR ADDING REQUIRED ONES

-- Alter Country
ALTER TABLE [Data].[Country]
ADD [CountryID] [int] NOT NULL DEFAULT (0);
GO

UPDATE [Data].[Country]
SET [CountryId] = 0; -- Placeholder; we'll update this after restoring data


ALTER TABLE [Data].[Country]
DROP COLUMN [CountryFlag], [FlagFileName], [FlagFileType];
GO 




Caution: Changing any part of an object name could break scripts and stored procedures.

Total execution time: 00:00:00.064

In [17]:
-- Alter Customer

EXEC sp_rename 'Data.Customer.PostCode', 'PostalCode', 'COLUMN';
GO

EXEC sp_rename 'Data.Customer.Town', 'City', 'COLUMN';
GO

EXEC sp_rename 'Data.Customer.Country', 'CountryId', 'COLUMN';
GO


Caution: Changing any part of an object name could break scripts and stored procedures.

Caution: Changing any part of an object name could break scripts and stored procedures.

Caution: Changing any part of an object name could break scripts and stored procedures.

Total execution time: 00:00:00.174

In [20]:
-- Alter Sales
EXEC sp_rename 'Data.Sales', 'Sale';
GO
EXEC sp_rename 'Data.Sale.TotalSalePrice', 'SaleTotal', 'COLUMN';
GO
EXEC sp_rename 'Data.Sale.SalesID', 'SaleID', 'COLUMN';
GO
ALTER TABLE [Data].[Sale]
DROP COLUMN [SaleDate], [ID];
GO 

: Msg 15225, Level 11, State 1, Procedure sp_rename, Line 436
No item by the name of 'Data.Sales' could be found in the current database 'PrestigeCars', given that @itemtype was input as '(null)'.

: Msg 15248, Level 11, State 1, Procedure sp_rename, Line 247
Either the parameter @objname is ambiguous or the claimed @objtype (COLUMN) is wrong.

Caution: Changing any part of an object name could break scripts and stored procedures.

: Msg 4924, Level 16, State 1, Line 8
ALTER TABLE DROP COLUMN failed because column 'SaleDate' does not exist in table 'Sale'.

Total execution time: 00:00:00.049

In [21]:
-- Alter SalesDetails
EXEC sp_rename 'Data.SalesDetails', 'SaleDetail';
GO
EXEC sp_rename 'Data.SaleDetail.SalesDetailsID', 'SaleDetailID', 'COLUMN';
GO
EXEC sp_rename 'Data.SaleDetail.SalesID', 'SaleID', 'COLUMN';
GO
ALTER TABLE [Data].[SaleDetail]
ADD [DateBought] DATE,
    [TimeBought] TIME;
GO

Caution: Changing any part of an object name could break scripts and stored procedures.

Caution: Changing any part of an object name could break scripts and stored procedures.

Caution: Changing any part of an object name could break scripts and stored procedures.

Commands completed successfully.

Total execution time: 00:00:00.103

In [23]:
-- Alter Stock

EXEC sp_rename 'Data.Stock.StockCode', 'StockID', 'COLUMN';
EXEC sp_rename 'Data.Stock.RepairsCost', 'RepairCost', 'COLUMN';
EXEC sp_rename 'Data.Stock.PartsCost', 'PartCost', 'COLUMN';
EXEC sp_rename 'Data.Stock.TransportInCost', 'TransportCost', 'COLUMN';
EXEC sp_rename 'Data.Stock.IsRHD', 'IsRightHandDrive', 'COLUMN';
GO

ALTER TABLE [Data].[Stock]
DROP COLUMN [BuyerComments],[DateBought],[TimeBought];
GO


Caution: Changing any part of an object name could break scripts and stored procedures.

Caution: Changing any part of an object name could break scripts and stored procedures.

Caution: Changing any part of an object name could break scripts and stored procedures.

Caution: Changing any part of an object name could break scripts and stored procedures.

Caution: Changing any part of an object name could break scripts and stored procedures.

Commands completed successfully.

Total execution time: 00:00:00.095

In [24]:
USE [PrestigeCars]
GO

/****** Object:  Schema [Udt]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE SCHEMA [Udt]
GO

/****** Object:  UserDefinedDataType [Udt].[SurrogateKeyInt]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[SurrogateKeyInt] FROM [int] NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[SalesRegion]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[SalesRegion] FROM [nvarchar](15) NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[CountryName]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[CountryName] FROM [nvarchar](150) NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[ISO2]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[ISO2] FROM [nchar](2) NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[ISO3]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[ISO3] FROM [nchar](3) NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[CustomerIDType]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[CustomerIDType] FROM [nvarchar](5) NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[CustomerNameType]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[CustomerNameType] FROM [nvarchar](150) NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[AddressType]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[AddressType] FROM [nvarchar](50) NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[TownType]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[CityType] FROM [nvarchar](50) NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[PostalCodeType]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[PostalCodeType] FROM [nvarchar](50) NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[BitType]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[BitType] FROM [bit] NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[ModelNameType]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[ModelNameType] FROM [nvarchar](150) NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[ModelVariantType]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[ModelVariantType] FROM [nvarchar](150) NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[YearType]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[YearType] FROM [char](4) NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[InvoiceNumberType]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[InvoiceNumberType] FROM [char](8) NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[MoneyType]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[SaleTotalType] FROM [numeric](18, 2) NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[TinyIntType]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[TinyIntType] FROM [tinyint] NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[StockIDType]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[StockIDType] FROM [nvarchar](50) NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[DateType]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[DateType] FROM [date] NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[TimeType]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[TimeType] FROM [time](7) NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[ColorType]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[ColorType] FROM [nvarchar](50) NOT NULL
GO

/****** Object:  UserDefinedDataType [Udt].[CommentType]    Script Date: 5/18/2025 02:33:00 PM ******/
CREATE TYPE [Udt].[CommentType] FROM [nvarchar](4000) NOT NULL
GO

Commands completed successfully.

: Msg 2714, Level 16, State 6, Line 5
There is already an object named 'Udt' in the database.

: Msg 2759, Level 16, State 0, Line 5
CREATE SCHEMA failed due to previous errors.

: Msg 219, Level 16, State 1, Line 9
The type 'Udt.SurrogateKeyInt' already exists, or you do not have permission to create it.

: Msg 219, Level 16, State 1, Line 13
The type 'Udt.SalesRegion' already exists, or you do not have permission to create it.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.183

In [26]:
ALTER TABLE [Data].[Country]
ADD [CountryId] [Udt].[SurrogateKeyInt] IDENTITY(1,1) NOT NULL;
GO

ALTER TABLE [Data].[Country]
ADD CONSTRAINT [PK_Country] PRIMARY KEY CLUSTERED 
(
    [CountryId] ASC
) WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY];
GO

ALTER TABLE [Data].[Country]
ALTER COLUMN [CountryName] [Udt].[CountryName] NOT NULL;
GO

ALTER TABLE [Data].[Country]
ALTER COLUMN [CountryISO2] [Udt].[ISO2] NOT NULL;
GO

ALTER TABLE [Data].[Country]
ALTER COLUMN [CountryISO3] [Udt].[ISO3] NOT NULL;
GO

ALTER TABLE [Data].[Country]
ALTER COLUMN [SalesRegionId] [Udt].[SurrogateKeyInt] NOT NULL;
GO

ALTER TABLE [Data].[Country]
ADD CONSTRAINT [DF_CountryId] DEFAULT (0) FOR [CountryId];
GO

ALTER TABLE [Data].[Country]
ADD CONSTRAINT [DF_CountryName] DEFAULT ('UNSPECIFIED') FOR [CountryName];
GO

ALTER TABLE [Data].[Country]
ADD CONSTRAINT [DF_CountryISO2] DEFAULT ('ZZ') FOR [CountryISO2];
GO

ALTER TABLE [Data].[Country]
ADD CONSTRAINT [DF_CountryISO3] DEFAULT ('ZZZ') FOR [CountryISO3];
GO

ALTER TABLE [Data].[Country] CHECK CONSTRAINT [FK_Country_SalesRegion]
GO

ALTER TABLE [Data].[Country] WITH CHECK ADD CONSTRAINT [CK_CountryISO2] CHECK (([CountryISO2] LIKE '[A-Z][A-Z]'))
GO

ALTER TABLE [Data].[Country] CHECK CONSTRAINT [CK_CountryISO2]
GO

ALTER TABLE [Data].[Country] WITH CHECK ADD CONSTRAINT [CK_CountryISO3] CHECK (([CountryISO3] LIKE '[A-Z][A-Z][A-Z]'))
GO

ALTER TABLE [Data].[Country] CHECK CONSTRAINT [CK_CountryISO3]
GO

CREATE UNIQUE NONCLUSTERED INDEX [UniqueCountryName_idx] ON [Data].[Country]
(
    [CountryName] ASC
) WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, IGNORE_DUP_KEY = OFF, DROP_EXISTING = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO

CREATE UNIQUE NONCLUSTERED INDEX [UniqueCountryISO3_idx] ON [Data].[Country]
(
    [CountryISO3] ASC
) WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, IGNORE_DUP_KEY = OFF, DROP_EXISTING = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO

: Msg 2705, Level 16, State 4, Line 1
Column names in each table must be unique. Column name 'CountryId' in table 'Data.Country' is specified more than once.

: Msg 1779, Level 16, State 0, Line 5
Table 'Country' already has a primary key defined on it.

: Msg 1750, Level 16, State 0, Line 5
Could not create constraint or index. See previous errors.

Commands completed successfully.

Commands completed successfully.

...

: Msg 1750, Level 16, State 1, Line 53
Could not create constraint or index. See previous errors.

Commands completed successfully.

: Msg 1913, Level 16, State 1, Line 59
The operation failed because an index or statistics with name 'UniqueCountryName_idx' already exists on table 'Data.Country'.

: Msg 1913, Level 16, State 1, Line 65
The operation failed because an index or statistics with name 'UniqueCountryISO3_idx' already exists on table 'Data.Country'.

Total execution time: 00:00:00.056

In [44]:
ALTER TABLE [Data].[Customer]
ADD [CountryId] [Udt].[SurrogateKeyInt] NOT NULL CONSTRAINT [DF_CountryId] DEFAULT (0);
GO

UPDATE [Data].[Customer]
SET [CountryId] = 0;

ALTER TABLE [Data].[Customer]
DROP COLUMN [Country];
GO

ALTER TABLE [Data].[Customer]
ALTER COLUMN [CustomerID] [nvarchar](5) NOT NULL;
GO

ALTER TABLE [Data].[Customer]
ALTER COLUMN [CustomerName] [Udt].[CustomerNameType] NOT NULL;
GO

ALTER TABLE [Data].[Customer]
ALTER COLUMN [Address1] [nvarchar](50) NOT NULL;
GO

ALTER TABLE [Data].[Customer]
ALTER COLUMN [Address2] [nvarchar](50) NOT NULL;
GO

ALTER TABLE [Data].[Customer]
ALTER COLUMN [City] [nvarchar](50) NOT NULL;
GO

ALTER TABLE [Data].[Customer]
ALTER COLUMN [PostalCode] [nvarchar](50) NOT NULL;
GO

ALTER TABLE [Data].[Customer]
ALTER COLUMN [IsReseller] [bit] NOT NULL;
GO

ALTER TABLE [Data].[Customer]
ALTER COLUMN [IsCreditRisk] [bit] NOT NULL;
GO

ALTER TABLE [Data].[Customer]
ADD CONSTRAINT [DF_CustomerID] DEFAULT ('00000') FOR [CustomerID];
GO

ALTER TABLE [Data].[Customer]
ADD CONSTRAINT [DF_CustomerName] DEFAULT ('UNSPECIFIED') FOR [CustomerName];
GO

ALTER TABLE [Data].[Customer]
ADD CONSTRAINT [DF_Address1] DEFAULT ('UNSPECIFIED') FOR [Address1];
GO

ALTER TABLE [Data].[Customer]
ADD CONSTRAINT [DF_Address2] DEFAULT ('UNSPECIFIED') FOR [Address2];
GO

ALTER TABLE [Data].[Customer]
ADD CONSTRAINT [DF_Town] DEFAULT ('UNSPECIFIED') FOR [Town];
GO

ALTER TABLE [Data].[Customer]
ADD CONSTRAINT [DF_PostalCode] DEFAULT ('UNSPECIFIED') FOR [PostalCode];
GO

ALTER TABLE [Data].[Customer]
ADD CONSTRAINT [DF_IsReseller] DEFAULT (0) FOR [IsReseller];
GO

ALTER TABLE [Data].[Customer]
ADD CONSTRAINT [DF_IsCreditRisk] DEFAULT (0) FOR [IsCreditRisk];
GO

ALTER TABLE [Data].[Customer] WITH CHECK ADD CONSTRAINT [FK_Customer_Country] FOREIGN KEY ([CountryId])
REFERENCES [Data].[CountryId] ([CountryId])
GO

ALTER TABLE [Data].[Customer] CHECK CONSTRAINT [FK_Customer_Country]
GO

ALTER TABLE [Data].[Customer] WITH CHECK ADD CONSTRAINT [CK_Customer_PostalCode] 
CHECK ((PostalCode = 'UNSPECIFIED' OR LEN(PostalCode) >= 5))
GO

ALTER TABLE [Data].[Customer] CHECK CONSTRAINT [CK_Customer_PostalCode]
GO

CREATE UNIQUE NONCLUSTERED INDEX [UniqueCustomerID_idx] ON [Data].[Customer]
(
    [CustomerID] ASC
) WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, IGNORE_DUP_KEY = OFF, DROP_EXISTING = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO

: Msg 2705, Level 16, State 4, Line 1
Column names in each table must be unique. Column name 'CountryId' in table 'Data.Customer' is specified more than once.

(0 rows affected)

: Msg 4924, Level 16, State 1, Line 8
ALTER TABLE DROP COLUMN failed because column 'Country' does not exist in table 'Customer'.

Commands completed successfully.

Commands completed successfully.

...

: Msg 2714, Level 16, State 5, Line 83
There is already an object named 'CK_Customer_PostalCode' in the database.

: Msg 1750, Level 16, State 1, Line 83
Could not create constraint or index. See previous errors.

Commands completed successfully.

: Msg 1913, Level 16, State 1, Line 90
The operation failed because an index or statistics with name 'UniqueCustomerID_idx' already exists on table 'Data.Customer'.

Total execution time: 00:00:00.131

In [29]:
-- data make
ALTER TABLE [Data].[Make]
ALTER COLUMN [MakeID] [Udt].[SurrogateKeyInt] NOT NULL;
GO

ALTER TABLE [Data].[Make]
ALTER COLUMN [MakeName] [Udt].[ModelNameType] NOT NULL;
GO

ALTER TABLE [Data].[Make]
ALTER COLUMN [CountryId] [Udt].[SurrogateKeyInt] NOT NULL;
GO

ALTER TABLE [Data].[Make]
ADD CONSTRAINT [DF_MakeID] DEFAULT (0) FOR [MakeID];
GO

ALTER TABLE [Data].[Make]
ADD CONSTRAINT [DF_MakeName] DEFAULT ('UNSPECIFIED') FOR [MakeName];
GO

ALTER TABLE [Data].[Make]
ADD CONSTRAINT [DF_CountryId] DEFAULT (0) FOR [CountryId];
GO

ALTER TABLE [Data].[Make] WITH CHECK ADD CONSTRAINT [FK_Make_Country] FOREIGN KEY ([CountryId])
REFERENCES [Data].[Country] ([CountryId])
GO

ALTER TABLE [Data].[Make] CHECK CONSTRAINT [FK_Make_Country]
GO

CREATE UNIQUE NONCLUSTERED INDEX [UniqueMakeName_idx] ON [Data].[Make]
(
    [MakeName] ASC
) WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, IGNORE_DUP_KEY = OFF, DROP_EXISTING = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

: Msg 1754, Level 16, State 0, Line 14
Defaults cannot be created on columns with an IDENTITY attribute. Table 'Make', column 'MakeID'.

: Msg 1750, Level 16, State 0, Line 14
Could not create constraint or index. See previous errors.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

: Msg 1913, Level 16, State 1, Line 33
The operation failed because an index or statistics with name 'UniqueMakeName_idx' already exists on table 'Data.Make'.

Total execution time: 00:00:00.385

In [30]:
ALTER TABLE [Data].[Model]
ALTER COLUMN [ModelID] [Udt].[SurrogateKeyInt] NOT NULL;
GO

ALTER TABLE [Data].[Model]
ALTER COLUMN [MakeID] [Udt].[SurrogateKeyInt] NOT NULL;
GO

ALTER TABLE [Data].[Model]
ALTER COLUMN [ModelName] [Udt].[ModelNameType] NOT NULL;
GO

ALTER TABLE [Data].[Model]
ALTER COLUMN [ModelVariant] [Udt].[ModelVariantType] NOT NULL;
GO

ALTER TABLE [Data].[Model]
ALTER COLUMN [YearFirstProduced] [Udt].[YearType] NOT NULL;
GO

ALTER TABLE [Data].[Model]
ALTER COLUMN [YearLastProduced] [Udt].[YearType] NOT NULL;
GO

ALTER TABLE [Data].[Model]
ADD CONSTRAINT [DF_ModelID] DEFAULT (0) FOR [ModelID];
GO

ALTER TABLE [Data].[Model]
ADD CONSTRAINT [DF_MakeID] DEFAULT (0) FOR [MakeID];
GO

ALTER TABLE [Data].[Model]
ADD CONSTRAINT [DF_ModelName] DEFAULT ('UNSPECIFIED') FOR [ModelName];
GO

ALTER TABLE [Data].[Model]
ADD CONSTRAINT [DF_ModelVariant] DEFAULT ('Standard') FOR [ModelVariant];
GO

ALTER TABLE [Data].[Model]
ADD CONSTRAINT [DF_YearFirstProduced] DEFAULT ('1900') FOR [YearFirstProduced];
GO

ALTER TABLE [Data].[Model]
ADD CONSTRAINT [DF_YearLastProduced] DEFAULT ('1900') FOR [YearLastProduced];
GO

ALTER TABLE [Data].[Model] WITH CHECK ADD CONSTRAINT [FK_Model_Make] FOREIGN KEY ([MakeID])
REFERENCES [Data].[Make] ([MakeID])
GO

ALTER TABLE [Data].[Model] CHECK CONSTRAINT [FK_Model_Make]
GO

ALTER TABLE [Data].[Model] WITH CHECK ADD CONSTRAINT [CK_Model_YearRange] 
CHECK ((YearLastProduced >= YearFirstProduced))
GO

ALTER TABLE [Data].[Model] CHECK CONSTRAINT [CK_Model_YearRange]
GO

ALTER TABLE [Data].[Model] WITH CHECK ADD CONSTRAINT [CK_Model_YearFirstProducedValid] 
CHECK ((TRY_CONVERT(INT, YearFirstProduced) BETWEEN 1900 AND YEAR(GETDATE()) + 1))
GO

ALTER TABLE [Data].[Model] CHECK CONSTRAINT [CK_Model_YearFirstProducedValid]
GO

ALTER TABLE [Data].[Model] WITH CHECK ADD CONSTRAINT [CK_Model_YearLastProducedValid] 
CHECK ((TRY_CONVERT(INT, YearLastProduced) BETWEEN 1900 AND YEAR(GETDATE()) + 10))
GO

ALTER TABLE [Data].[Model] CHECK CONSTRAINT [CK_Model_YearLastProducedValid]
GO

CREATE UNIQUE NONCLUSTERED INDEX [UniqueModelName_idx] ON [Data].[Model]
(
    [ModelName] ASC,
    [MakeID] ASC
) WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, IGNORE_DUP_KEY = OFF, DROP_EXISTING = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

: Msg 2714, Level 16, State 5, Line 70
There is already an object named 'CK_Model_YearLastProducedValid' in the database.

: Msg 1750, Level 16, State 1, Line 70
Could not create constraint or index. See previous errors.

Commands completed successfully.

: Msg 1913, Level 16, State 1, Line 77
The operation failed because an index or statistics with name 'UniqueModelName_idx' already exists on table 'Data.Model'.

Total execution time: 00:00:00.137

In [31]:
-- stock
ALTER TABLE [Data].[Stock]
ALTER COLUMN [StockID] [nvarchar](50) NOT NULL;
GO

ALTER TABLE [Data].[Stock]
ALTER COLUMN [ModelID] [Udt].[SurrogateKeyInt] NOT NULL;
GO

ALTER TABLE [Data].[Stock]
ALTER COLUMN [Cost] [money] NOT NULL;
GO

ALTER TABLE [Data].[Stock]
ALTER COLUMN [RepairCost] [money] NOT NULL;
GO

ALTER TABLE [Data].[Stock]
ALTER COLUMN [PartCost] [money] NOT NULL;
GO

ALTER TABLE [Data].[Stock]
ALTER COLUMN [TransportCost] [money] NOT NULL;
GO

ALTER TABLE [Data].[Stock]
ALTER COLUMN [IsRHD] [bit] NOT NULL;
GO

ALTER TABLE [Data].[Stock]
ALTER COLUMN [Color] [nvarchar](50) NOT NULL;
GO

ALTER TABLE [Data].[Stock]
ALTER COLUMN [BuyerComment] [nvarchar](4000) NOT NULL;
GO

ALTER TABLE [Data].[Stock]
ALTER COLUMN [DateBought] [date] NOT NULL;
GO

ALTER TABLE [Data].[Stock]
ALTER COLUMN [TimeBought] [time](7) NOT NULL;
GO

ALTER TABLE [Data].[Stock]
ADD CONSTRAINT [DF_StockID] DEFAULT (NEWID()) FOR [StockID];
GO

ALTER TABLE [Data].[Stock]
ADD CONSTRAINT [DF_Cost] DEFAULT (0) FOR [Cost];
GO

ALTER TABLE [Data].[Stock]
ADD CONSTRAINT [DF_RepairCost] DEFAULT (0) FOR [RepairCost];
GO

ALTER TABLE [Data].[Stock]
ADD CONSTRAINT [DF_PartCost] DEFAULT (0) FOR [PartCost];
GO

ALTER TABLE [Data].[Stock]
ADD CONSTRAINT [DF_TransportCost] DEFAULT (0) FOR [TransportCost];
GO

ALTER TABLE [Data].[Stock]
ADD CONSTRAINT [DF_IsRHD] DEFAULT (0) FOR [IsRHD];
GO

ALTER TABLE [Data].[Stock]
ADD CONSTRAINT [DF_Color] DEFAULT ('UNSPECIFIED') FOR [Color];
GO

ALTER TABLE [Data].[Stock]
ADD CONSTRAINT [DF_BuyerComment] DEFAULT ('UNSPECIFIED') FOR [BuyerComment];
GO

ALTER TABLE [Data].[Stock]
ADD CONSTRAINT [DF_DateBought] DEFAULT (GETDATE()) FOR [DateBought];
GO

ALTER TABLE [Data].[Stock]
ADD CONSTRAINT [DF_TimeBought] DEFAULT ('00:00:00') FOR [TimeBought];
GO

ALTER TABLE [Data].[Stock] WITH CHECK ADD CONSTRAINT [FK_Stock_Model] FOREIGN KEY ([ModelID])
REFERENCES [Data].[Model] ([ModelID])
GO

ALTER TABLE [Data].[Stock] CHECK CONSTRAINT [FK_Stock_Model]
GO

ALTER TABLE [Data].[Stock] WITH CHECK ADD CONSTRAINT [CK_Stock_Cost] 
CHECK ((Cost >= 0 AND RepairCost >= 0 AND PartCost >= 0 AND TransportCost >= 0))
GO

ALTER TABLE [Data].[Stock] CHECK CONSTRAINT [CK_Stock_Cost]
GO

CREATE UNIQUE NONCLUSTERED INDEX [UniqueStockID_idx] ON [Data].[Stock]
(
    [StockID] ASC
) WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, IGNORE_DUP_KEY = OFF, DROP_EXISTING = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.122

In [33]:
-- sales
ALTER TABLE [Data].[Sale]
ALTER COLUMN [SaleID] [Udt].[SurrogateKeyInt] NOT NULL;
GO

ALTER TABLE [Data].[Sale]
ALTER COLUMN [CustomerID] [nvarchar](5) NOT NULL;
GO

ALTER TABLE [Data].[Sale]
ALTER COLUMN [InvoiceNumber] [char](8) NOT NULL;
GO

ALTER TABLE [Data].[Sale]
ALTER COLUMN [SaleTotal] [numeric](18, 2) NOT NULL;
GO

ALTER TABLE [Data].[Sale]
ADD CONSTRAINT [DF_SaleID] DEFAULT (0) FOR [SaleID];
GO

ALTER TABLE [Data].[Sale]
ADD CONSTRAINT [DF_CustomerID] DEFAULT ('00000') FOR [CustomerID];
GO

ALTER TABLE [Data].[Sale]
ADD CONSTRAINT [DF_InvoiceNumber] DEFAULT ('00000000') FOR [InvoiceNumber];
GO

ALTER TABLE [Data].[Sale]
ADD CONSTRAINT [DF_SaleTotal] DEFAULT (0) FOR [SaleTotal];
GO

ALTER TABLE [Data].[Sale] WITH CHECK ADD CONSTRAINT [FK_Sales_Customer] FOREIGN KEY ([CustomerID])
REFERENCES [Data].[Customer] ([CustomerID])
GO

ALTER TABLE [Data].[Sale] CHECK CONSTRAINT [FK_Sales_Customer]
GO

ALTER TABLE [Data].[Sale] WITH CHECK ADD CONSTRAINT [CK_Sales_SaleTotal] 
CHECK ((SaleTotal >= 0))
GO

ALTER TABLE [Data].[Sale] CHECK CONSTRAINT [CK_Sales_SaleTotal]
GO

CREATE UNIQUE NONCLUSTERED INDEX [UniqueSaleID_idx] ON [Data].[Sale]
(
    [SaleID] ASC
) WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, IGNORE_DUP_KEY = OFF, DROP_EXISTING = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO

CREATE UNIQUE NONCLUSTERED INDEX [UniqueInvoiceNumber_idx] ON [Data].[Sale]
(
    [InvoiceNumber] ASC
) WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, IGNORE_DUP_KEY = OFF, DROP_EXISTING = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.081

In [34]:
-- sale detail
ALTER TABLE [Data].[SaleDetail]
ALTER COLUMN [SaleDetailID] [Udt].[SurrogateKeyInt] NOT NULL;
GO

ALTER TABLE [Data].[SaleDetail]
ALTER COLUMN [SaleID] [Udt].[SurrogateKeyInt] NOT NULL;
GO

ALTER TABLE [Data].[SaleDetail]
ALTER COLUMN [LineItemNumber] [tinyint] NOT NULL;
GO

ALTER TABLE [Data].[SaleDetail]
ALTER COLUMN [StockID] [nvarchar](50) NOT NULL;
GO

ALTER TABLE [Data].[SaleDetail]
ALTER COLUMN [SalePrice] [numeric](18, 2) NOT NULL;
GO

ALTER TABLE [Data].[SaleDetail]
ALTER COLUMN [LineItemDiscount] [numeric](18, 2) NOT NULL;
GO

ALTER TABLE [Data].[SaleDetail]
ALTER COLUMN [DateBought] [date] NOT NULL;
GO

ALTER TABLE [Data].[SaleDetail]
ALTER COLUMN [TimeBought] [time](7) NOT NULL;
GO

ALTER TABLE [Data].[SaleDetail]
ADD CONSTRAINT [DF_SaleDetailID] DEFAULT (0) FOR [SaleDetailID];
GO

ALTER TABLE [Data].[SaleDetail]
ADD CONSTRAINT [DF_SaleID] DEFAULT (0) FOR [SaleID];
GO

ALTER TABLE [Data].[SaleDetail]
ADD CONSTRAINT [DF_LineItemNumber] DEFAULT (0) FOR [LineItemNumber];
GO

ALTER TABLE [Data].[SaleDetail]
ADD CONSTRAINT [DF_StockID] DEFAULT (NEWID()) FOR [StockID];
GO

ALTER TABLE [Data].[SaleDetail]
ADD CONSTRAINT [DF_SalePrice] DEFAULT (0) FOR [SalePrice];
GO

ALTER TABLE [Data].[SaleDetail]
ADD CONSTRAINT [DF_LineItemDiscount] DEFAULT (0) FOR [LineItemDiscount];
GO

ALTER TABLE [Data].[SaleDetail]
ADD CONSTRAINT [DF_DateBought] DEFAULT (GETDATE()) FOR [DateBought];
GO

ALTER TABLE [Data].[SaleDetail]
ADD CONSTRAINT [DF_TimeBought] DEFAULT ('00:00:00') FOR [TimeBought];
GO

ALTER TABLE [Data].[SaleDetail] WITH CHECK ADD CONSTRAINT [FK_SalesDetails_Sales] FOREIGN KEY ([SaleID])
REFERENCES [Data].[Sale] ([SaleID])
GO

ALTER TABLE [Data].[SaleDetail] CHECK CONSTRAINT [FK_SalesDetails_Sales]
GO

ALTER TABLE [Data].[SaleDetail] WITH CHECK ADD CONSTRAINT [FK_SalesDetails_Stock] FOREIGN KEY ([StockID])
REFERENCES [Data].[Stock] ([StockID])
GO

ALTER TABLE [Data].[SaleDetail] CHECK CONSTRAINT [FK_SalesDetails_Stock]
GO

ALTER TABLE [Data].[SaleDetail] WITH CHECK ADD CONSTRAINT [CK_SalesDetails_Prices] 
CHECK ((SalePrice >= 0 AND LineItemDiscount >= 0))
GO

ALTER TABLE [Data].[SaleDetail] CHECK CONSTRAINT [CK_SalesDetails_Prices]
GO

ALTER TABLE [Data].[SaleDetail] WITH CHECK ADD CONSTRAINT [CK_SalesDetails_LineItemNumber] 
CHECK ((LineItemNumber > 0))
GO

ALTER TABLE [Data].[SaleDetail] CHECK CONSTRAINT [CK_SalesDetails_LineItemNumber]
GO

CREATE UNIQUE NONCLUSTERED INDEX [UniqueSaleDetailID_idx] ON [Data].[SaleDetail]
(
    [SaleDetailID] ASC
) WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, IGNORE_DUP_KEY = OFF, DROP_EXISTING = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.176

In [45]:
INSERT INTO [Data].[Customer] (CustomerID, CustomerName, Address1, Address2, Town, PostalCode, CountryId, IsReseller, IsCreditRisk)
SELECT 
    COALESCE(BC.CustomerID, '00000'),
    COALESCE(BC.CustomerName, 'UNSPECIFIED'),
    COALESCE(BC.Address1, 'UNSPECIFIED'),
    COALESCE(BC.Address2, 'UNSPECIFIED'),
    COALESCE(BC.Town, 'UNSPECIFIED'),
    COALESCE(BC.PostalCode, 'UNSPECIFIED'),
    COALESCE(C.CountryId, 0),
    COALESCE(BC.IsReseller, 0),
    COALESCE(BC.IsCreditRisk, 0)
FROM [BackupData].[Customer] BC
LEFT JOIN [Data].[Country] C ON TRIM(BC.Country) = TRIM(C.CountryISO2);

(88 rows affected)

Total execution time: 00:00:00.090

In [54]:
USE [PrestigeCars]
GO

-- Insert into [Data].[Country] from [BackupData].[Country]
INSERT INTO [Data].[Country] (CountryName, CountryISO2, CountryISO3, SalesRegion)
SELECT 
    COALESCE(LEFT(TRIM(BC.CountryName), 150), 'UNSPECIFIED'),
    COALESCE(LEFT(TRIM(BC.CountryISO2), 2), 'ZZ'),
    COALESCE(LEFT(TRIM(BC.CountryISO3), 3), 'ZZZ'),
    COALESCE(LEFT(TRIM(BC.SaleRegion), 3), 'UNK')
FROM [BackupData].[Country] BC;
GO

Commands completed successfully.

(10 rows affected)

Total execution time: 00:00:00.099

In [67]:
-- insert into make
SET IDENTITY_INSERT [Data].[Make] ON;
GO

INSERT INTO [Data].[Make] (MakeID, MakeName, CountryId)
SELECT 
    BM.MakeID,
    COALESCE(LEFT(TRIM(BM.MakeName), 150), 'UNSPECIFIED'),
    C.CountryId
FROM [BackupData].[Make] BM
INNER JOIN [Data].[Country] C ON TRIM(BM.MakeCountry) = TRIM(C.CountryISO3);
GO

SET IDENTITY_INSERT [Data].[Make] OFF;
GO

Commands completed successfully.

(20 rows affected)

Commands completed successfully.

Total execution time: 00:00:00.034

In [92]:
SET IDENTITY_INSERT [Data].[Make] ON;

WITH RankedMakes AS (
    SELECT 
        MakeID,
        MakeName,
        MakeCountry,
        ROW_NUMBER() OVER (PARTITION BY MakeID ORDER BY MakeID) AS RN
    FROM [BackupData].[Make]
)
INSERT INTO [Data].[Make] (MakeID, MakeName, CountryId)
SELECT 
    RM.MakeID,
    COALESCE(LEFT(TRIM(RM.MakeName), 150), 'UNSPECIFIED'),
    COALESCE(C.CountryId, 0)
FROM RankedMakes RM
LEFT JOIN [Data].[Country] C ON TRIM(RM.MakeCountry) = TRIM(C.CountryISO3)
WHERE RM.RN = 1;

SET IDENTITY_INSERT [Data].[Make] OFF;

(26 rows affected)

Total execution time: 00:00:00.028

In [93]:
USE [PrestigeCars]
GO

-- Drop foreign key constraints referencing [Data].[Model]
ALTER TABLE [Data].[Stock] DROP CONSTRAINT IF EXISTS [FK_Stock_Model];
GO

-- Drop existing index on [Data].[Model] if it exists
IF EXISTS (SELECT * FROM sys.indexes WHERE name = 'UniqueModelName_idx' AND object_id = OBJECT_ID('[Data].[Model]'))
BEGIN
    DROP INDEX [UniqueModelName_idx] ON [Data].[Model];
END
GO

-- Drop the primary key constraint to allow truncation
IF EXISTS (SELECT * FROM sys.indexes WHERE name = 'PK_Model' AND object_id = OBJECT_ID('[Data].[Model]'))
BEGIN
    ALTER TABLE [Data].[Model] DROP CONSTRAINT [PK_Model];
END
GO

-- Truncate [Data].[Model] to start fresh
TRUNCATE TABLE [Data].[Model];
GO

-- Recreate the primary key constraint
ALTER TABLE [Data].[Model]
ADD CONSTRAINT [PK_Model] PRIMARY KEY CLUSTERED ([ModelID] ASC);
GO

-- Set IDENTITY_INSERT ON for [Data].[Model]
SET IDENTITY_INSERT [Data].[Model] ON;
GO

-- Insert data, handling duplicates in ModelID and ModelName/MakeID
WITH RankedModels AS (
    SELECT 
        ModelID,
        MakeID,
        ModelName,
        ModelVariant,
        YearFirstProduced,
        YearLastProduced,
        ROW_NUMBER() OVER (PARTITION BY ModelID ORDER BY ModelID) AS RN_ModelID,
        ROW_NUMBER() OVER (PARTITION BY ModelName, MakeID ORDER BY ModelID) AS RN_ModelName_MakeID
    FROM [BackupData].[Model]
)
INSERT INTO [Data].[Model] (ModelID, MakeID, ModelName, ModelVariant, YearFirstProduced, YearLastProduced)
SELECT 
    RM.ModelID,
    COALESCE(M.MakeID, 0),
    COALESCE(LEFT(TRIM(RM.ModelName), 150), 'UNSPECIFIED'),
    COALESCE(LEFT(TRIM(RM.ModelVariant), 150), 'Standard'),
    COALESCE(LEFT(TRIM(RM.YearFirstProduced), 4), '1900'),
    COALESCE(LEFT(TRIM(RM.YearLastProduced), 4), '1900')
FROM RankedModels RM
LEFT JOIN [Data].[Make] M ON RM.MakeID = M.MakeID
WHERE RM.RN_ModelID = 1  -- Handle duplicates in ModelID
  AND RM.RN_ModelName_MakeID = 1;  -- Handle duplicates in ModelName/MakeID
GO

-- Turn IDENTITY_INSERT OFF
SET IDENTITY_INSERT [Data].[Model] OFF;
GO

-- Recreate the unique index
CREATE UNIQUE NONCLUSTERED INDEX [UniqueModelName_idx] ON [Data].[Model]
(
    [ModelName] ASC,
    [MakeID] ASC
) WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, IGNORE_DUP_KEY = OFF, DROP_EXISTING = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO

-- Recreate the foreign key constraint
ALTER TABLE [Data].[Stock] WITH CHECK ADD CONSTRAINT [FK_Stock_Model] FOREIGN KEY ([ModelID])
REFERENCES [Data].[Model] ([ModelID]);
GO

ALTER TABLE [Data].[Stock] CHECK CONSTRAINT [FK_Stock_Model];
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.217

In [112]:
USE [PrestigeCars]
GO

-- Insert data from backup, avoiding duplicate StockIDs and ensuring ModelID exists
WITH RankedStock AS (
    SELECT 
        StockID,
        ModelID,
        Cost,
        RepairCost,
        PartCost,
        TransportCost,
        IsRHD AS IsRightHandDrive, -- Map IsRHD to IsRightHandDrive
        Color,
        ROW_NUMBER() OVER (PARTITION BY StockID ORDER BY StockID) AS RN
    FROM [BackupData].[Stock]
)
INSERT INTO [Data].[Stock] (
    StockID,
    ModelID,
    Cost,
    RepairCost,
    PartCost,
    TransportCost,
    IsRightHandDrive,
    Color
)
SELECT 
    COALESCE(NULLIF(LTRIM(RTRIM(RS.StockID)), ''), NEWID()),           -- Cleaned or generated StockID
    RS.ModelID,                                                         -- Assumed valid due to INNER JOIN
    COALESCE(RS.Cost, 0),
    COALESCE(RS.RepairCost, 0),
    COALESCE(RS.PartCost, 0),
    COALESCE(RS.TransportCost, 0),
    COALESCE(RS.IsRightHandDrive, 0),
    COALESCE(LEFT(TRIM(RS.Color), 50), 'UNSPECIFIED')
FROM RankedStock RS
INNER JOIN [Data].[Model] M ON RS.ModelID = M.ModelID -- Ensures valid FK
WHERE RS.RN = 1;
GO

-- Create the unique index on StockID
CREATE UNIQUE NONCLUSTERED INDEX [UniqueStockID_idx] ON [Data].[Stock] (
    [StockID] ASC
) WITH (
    PAD_INDEX = OFF, 
    STATISTICS_NORECOMPUTE = OFF, 
    SORT_IN_TEMPDB = OFF, 
    IGNORE_DUP_KEY = OFF, 
    DROP_EXISTING = OFF, 
    ONLINE = OFF, 
    ALLOW_ROW_LOCKS = ON, 
    ALLOW_PAGE_LOCKS = ON, 
    OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF
) ON [PRIMARY];
GO

Commands completed successfully.

(391 rows affected)

: Msg 1913, Level 16, State 1, Line 43
The operation failed because an index or statistics with name 'UniqueStockID_idx' already exists on table 'Data.Stock'.

Total execution time: 00:00:00.064

In [129]:
USE [PrestigeCars]
GO

-- Step 1: Drop and recreate [Data].[SaleDetail]
IF OBJECT_ID('Data.SaleDetail') IS NOT NULL
    DROP TABLE [Data].[SaleDetail];
GO

-- Step 2: Create table
CREATE TABLE [Data].[SaleDetail]
(
    SaleDetailID [Udt].[SurrogateKeyInt] NOT NULL,
    SaleID [Udt].[SurrogateKeyInt] NOT NULL,
    LineItemNumber [Udt].[TinyIntType] NOT NULL,
    StockID [Udt].[StockIDType] NOT NULL,
    SalePrice [Udt].[MoneyType] NOT NULL,
    LineItemDiscount [Udt].[MoneyType] NOT NULL,
    DateBought [Udt].[DateType] NOT NULL,
    TimeBought [Udt].[TimeType] NOT NULL,
    CONSTRAINT [PK_SaleDetail] PRIMARY KEY CLUSTERED ([SaleDetailID] ASC),
    CONSTRAINT [FK_SaleDetail_Sales] FOREIGN KEY ([SaleID]) REFERENCES [Data].[Sales] ([SaleID]),
    CONSTRAINT [FK_SaleDetail_Stock] FOREIGN KEY ([StockID]) REFERENCES [Data].[Stock] ([StockID])
);
GO

-- Step 3: Declare and insert default StockID (in same batch)
DECLARE @DefaultStockID NVARCHAR(50) = CAST(NEWID() AS NVARCHAR(50));

IF NOT EXISTS (SELECT 1 FROM [Data].[Stock] WHERE StockID = @DefaultStockID)
BEGIN
    INSERT INTO [Data].[Stock] (StockID, ModelID, Cost, RepairCost, PartCost, TransportCost, IsRightHandDrive, Color)
    VALUES (@DefaultStockID, 0, 0, 0, 0, 0, 0, 'UNSPECIFIED');
END;

-- Step 4: Insert data into SaleDetail
;WITH RankedSalesDetails AS (
    SELECT 
        SaleDetailID,
        SaleID,
        LineItemNumber,
        StockID,
        SalePrice,
        LineItemDiscount,
        DateBought,
        TimeBought,
        ROW_NUMBER() OVER (PARTITION BY SaleDetailID ORDER BY SaleDetailID) AS RN
    FROM [BackupData].[SaleDetail]
)
INSERT INTO [Data].[SaleDetail] (
    SaleDetailID, SaleID, LineItemNumber, StockID, 
    SalePrice, LineItemDiscount, DateBought, TimeBought)
SELECT 
    RS.SaleDetailID,
    COALESCE(S.SaleID, 0),
    COALESCE(RS.LineItemNumber, 0),
    COALESCE(ST.StockID, @DefaultStockID),
    COALESCE(RS.SalePrice, 0),
    COALESCE(RS.LineItemDiscount, 0),
    COALESCE(RS.DateBought, GETDATE()),
    COALESCE(RS.TimeBought, '00:00:00')
FROM RankedSalesDetails RS
LEFT JOIN [Data].[Sales] S ON RS.SaleID = S.SaleID
LEFT JOIN [Data].[Stock] ST ON LEFT(TRIM(RS.StockID), 50) = LEFT(TRIM(ST.StockID), 50)
WHERE RS.RN = 1;
GO

-- Step 5: Drop and recreate index if necessary
IF EXISTS (
    SELECT * 
    FROM sys.indexes 
    WHERE name = 'UniqueSaleDetailID_idx' 
    AND object_id = OBJECT_ID('Data.SaleDetail')
)
DROP INDEX [UniqueSaleDetailID_idx] ON [Data].[SaleDetail];
GO

CREATE UNIQUE NONCLUSTERED INDEX [UniqueSaleDetailID_idx] ON [Data].[SaleDetail] ([SaleDetailID] ASC)
WITH (
    PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF,
    IGNORE_DUP_KEY = OFF, DROP_EXISTING = OFF, ONLINE = OFF,
    ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF
) ON [PRIMARY];
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(1 row affected)

(351 rows affected)

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.332

In [128]:
USE [PrestigeCars];
GO

-- Ensure default customer exists
IF NOT EXISTS (SELECT 1 FROM [Data].[Customer] WHERE CustomerID = '00000')
BEGIN
    INSERT INTO [Data].[Customer] (CustomerID, CountryID, CustomerName)
    VALUES ('00000', '000', 'UNKNOWN');
END
GO

-- Drop existing FK on SaleDetail -> Sale (if present)
IF EXISTS (
    SELECT * FROM sys.foreign_keys 
    WHERE name = 'FK_SaleDetail_Sales' AND parent_object_id = OBJECT_ID('[Data].[SaleDetail]')
)
BEGIN
    ALTER TABLE [Data].[SaleDetail] DROP CONSTRAINT [FK_SaleDetail_Sales];
END
GO

-- Drop and recreate [Data].[Sale]
IF OBJECT_ID('[Data].[Sale]', 'U') IS NOT NULL
BEGIN
    DROP TABLE [Data].[Sale];
END
GO

CREATE TABLE [Data].[Sale]
(
    SaleID [Udt].[SurrogateKeyInt] NOT NULL,
    CustomerID [Udt].[CustomerIDType] NOT NULL,
    InvoiceNumber [Udt].[InvoiceNumberType] NOT NULL,
    SaleTotal [Udt].[MoneyType] NOT NULL,
    CONSTRAINT [PK_Sale] PRIMARY KEY CLUSTERED ([SaleID]),
    CONSTRAINT [FK_Sales_Customer] FOREIGN KEY ([CustomerID]) REFERENCES [Data].[Customer] ([CustomerID])
);
GO

-- Insert default sale
IF NOT EXISTS (SELECT 1 FROM [Data].[Sale] WHERE SaleID = 0)
BEGIN
    INSERT INTO [Data].[Sale] (SaleID, CustomerID, InvoiceNumber, SaleTotal)
    VALUES (0, '00000', '00000000', 0);
END
GO

-- Insert deduplicated and validated sales from backup
WITH RankedSales AS (
    SELECT 
        SaleID,
        CustomerID,
        InvoiceNumber,
        SaleTotal,
        ROW_NUMBER() OVER (PARTITION BY SaleID ORDER BY SaleID) AS RN
    FROM [BackupData].[Sale]
)
INSERT INTO [Data].[Sale] (SaleID, CustomerID, InvoiceNumber, SaleTotal)
SELECT 
    RS.SaleID,
    CASE 
        WHEN C.CustomerID IS NOT NULL THEN C.CustomerID 
        ELSE '00000' 
    END,
    COALESCE(LEFT(TRIM(RS.InvoiceNumber), 8), '00000000'),
    COALESCE(RS.SaleTotal, 0)
FROM RankedSales RS
LEFT JOIN [Data].[Customer] C ON LEFT(TRIM(RS.CustomerID), 5) = LEFT(TRIM(C.CustomerID), 5)
WHERE RS.RN = 1;
GO

-- Recreate indexes
IF EXISTS (SELECT * FROM sys.indexes WHERE name = 'UniqueSaleID_idx' AND object_id = OBJECT_ID('[Data].[Sale]'))
BEGIN
    DROP INDEX [UniqueSaleID_idx] ON [Data].[Sale];
END

CREATE UNIQUE NONCLUSTERED INDEX [UniqueSaleID_idx] ON [Data].[Sale] (SaleID);
GO

IF EXISTS (SELECT * FROM sys.indexes WHERE name = 'UniqueInvoiceNumber_idx' AND object_id = OBJECT_ID('[Data].[Sale]'))
BEGIN
    DROP INDEX [UniqueInvoiceNumber_idx] ON [Data].[Sale];
END

CREATE UNIQUE NONCLUSTERED INDEX [UniqueInvoiceNumber_idx] ON [Data].[Sale] (InvoiceNumber);
GO

-- Recreate FK constraint (only after successful insert)
ALTER TABLE [Data].[SaleDetail] WITH CHECK 
ADD CONSTRAINT [FK_SaleDetail_Sales] FOREIGN KEY ([SaleID]) REFERENCES [Data].[Sale] ([SaleID]);

ALTER TABLE [Data].[SaleDetail] CHECK CONSTRAINT [FK_SaleDetail_Sales];
GO

Commands completed successfully.

(1 row affected)

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

(324 rows affected)

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.238

In [130]:
-- okay turn datatransfer.sales2015 into a view

CREATE FUNCTION [DataTransfer].[GetSalesByYear]
(
    @Year INT
)
RETURNS TABLE
AS
RETURN
(
    SELECT *, 2015 AS SalesYear FROM [DataTransfer].[Sales2015] WHERE @Year = 2015
    UNION ALL
    SELECT *, 2016 AS SalesYear FROM [DataTransfer].[Sales2016] WHERE @Year = 2016
    UNION ALL
    SELECT *, 2017 AS SalesYear FROM [DataTransfer].[Sales2017] WHERE @Year = 2017
    UNION ALL
    SELECT *, 2018 AS SalesYear FROM [DataTransfer].[Sales2018] WHERE @Year = 2018
);


Commands completed successfully.

Total execution time: 00:00:00.041

In [131]:
-- turn pivot table into a view
CREATE VIEW [Data].[ColorSalesByYear]
AS
SELECT 
    s.Color,
    SUM(CASE WHEN YEAR(sd.DateBought) = 2015 THEN sd.SalePrice ELSE 0 END) AS [2015],
    SUM(CASE WHEN YEAR(sd.DateBought) = 2016 THEN sd.SalePrice ELSE 0 END) AS [2016],
    SUM(CASE WHEN YEAR(sd.DateBought) = 2017 THEN sd.SalePrice ELSE 0 END) AS [2017],
    SUM(CASE WHEN YEAR(sd.DateBought) = 2018 THEN sd.SalePrice ELSE 0 END) AS [2018]
FROM [Data].[SaleDetail] sd
INNER JOIN [Data].[Stock] s ON sd.StockID = s.StockID
GROUP BY s.Color;

Commands completed successfully.

Total execution time: 00:00:00.028

In [133]:
-- flatten script for reference 

SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

-- =============================================
-- Author:        Jaskaran Bains
-- Create date:   2025-05-20
-- Description:   Creates a full backup of all tables in the [Reference] schema into [BackupReference]
-- Object type:   Stored Procedure
-- Usage:         EXEC [BackupReference].[BackupReferenceSchema] @UserAuthorizationKey = 1;
-- =============================================
CREATE PROCEDURE [BackupReference].[BackupReferenceSchema]
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Create BackupReference schema if not exists
    IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'BackupReference')
    BEGIN
        EXEC('CREATE SCHEMA BackupReference');
    END

    -- Backup each Reference table
    SELECT * INTO [BackupReference].[Budget] FROM [Reference].[Budget];
    SELECT * INTO [BackupReference].[Forex] FROM [Reference].[Forex];
    SELECT * INTO [BackupReference].[MarketingCategories] FROM [Reference].[MarketingCategories];
    SELECT * INTO [BackupReference].[MarketingInformation] FROM [Reference].[MarketingInformation];
    SELECT * INTO [BackupReference].[SalesBudgets] FROM [Reference].[SalesBudgets];
    SELECT * INTO [BackupReference].[SalesCategory] FROM [Reference].[SalesCategory];
    SELECT * INTO [BackupReference].[Staff] FROM [Reference].[Staff];
    SELECT * INTO [BackupReference].[StaffHierarchy] FROM [Reference].[StaffHierarchy];
    SELECT * INTO [BackupReference].[YearlySales] FROM [Reference].[YearlySales];

    -- Log row count for the last insert (or modify if you want to count all)
    DECLARE @WorkFlowStepTableRowCount INT = @@ROWCOUNT;

    -- Log to workflow tracker
    EXEC [Group1].[TrackWorkFlow]
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Backed up all of [Reference] to [BackupReference]',
        @WorkFlowStepTableRowCount = @WorkFlowStepTableRowCount,
        @UserAuthorizationKey = @UserAuthorizationKey;

END;
GO



Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.046

In [136]:
--complete truncation and or table dropping script for reference 
USE [PrestigeCars];
GO

-- Drop Reference tables in the correct dependency order (child to parent if needed)
IF OBJECT_ID('[Reference].[YearlySales]', 'U') IS NOT NULL
    DROP TABLE [Reference].[YearlySales];
GO

IF OBJECT_ID('[Reference].[StaffHierarchy]', 'U') IS NOT NULL
    DROP TABLE [Reference].[StaffHierarchy];
GO

IF OBJECT_ID('[Reference].[Staff]', 'U') IS NOT NULL
    DROP TABLE [Reference].[Staff];
GO

IF OBJECT_ID('[Reference].[SalesCategory]', 'U') IS NOT NULL
    DROP TABLE [Reference].[SalesCategory];
GO

IF OBJECT_ID('[Reference].[SalesBudgets]', 'U') IS NOT NULL
    DROP TABLE [Reference].[SalesBudgets];
GO

IF OBJECT_ID('[Reference].[MarketingInformation]', 'U') IS NOT NULL
    DROP TABLE [Reference].[MarketingInformation];
GO

IF OBJECT_ID('[Reference].[MarketingCategories]', 'U') IS NOT NULL
    DROP TABLE [Reference].[MarketingCategories];
GO

IF OBJECT_ID('[Reference].[Forex]', 'U') IS NOT NULL
    DROP TABLE [Reference].[Forex];
GO

IF OBJECT_ID('[Reference].[Budget]', 'U') IS NOT NULL
    DROP TABLE [Reference].[Budget];
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.023

In [137]:
-- turn all of refrence into views
USE [PrestigeCars];
GO

-- Budget
CREATE VIEW [Reference].[Budget]
AS
SELECT * FROM [BackupReference].[Budget];
GO

-- Forex
CREATE VIEW [Reference].[Forex]
AS
SELECT * FROM [BackupReference].[Forex];
GO

-- MarketingCategories
CREATE VIEW [Reference].[MarketingCategories]
AS
SELECT * FROM [BackupReference].[MarketingCategories];
GO

-- MarketingInformation
CREATE VIEW [Reference].[MarketingInformation]
AS
SELECT * FROM [BackupReference].[MarketingInformation];
GO

-- SalesBudgets
CREATE VIEW [Reference].[SalesBudgets]
AS
SELECT * FROM [BackupReference].[SalesBudgets];
GO

-- SalesCategory
CREATE VIEW [Reference].[SalesCategory]
AS
SELECT * FROM [BackupReference].[SalesCategory];
GO

-- Staff
CREATE VIEW [Reference].[Staff]
AS
SELECT * FROM [BackupReference].[Staff];
GO

-- StaffHierarchy
CREATE VIEW [Reference].[StaffHierarchy]
AS
SELECT * FROM [BackupReference].[StaffHierarchy];
GO

-- YearlySales
CREATE VIEW [Reference].[YearlySales]
AS
SELECT * FROM [BackupReference].[YearlySales];
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.070